In [1]:
import time

import torch
from torch_sparse import SparseTensor

t = torch.arange(10, 14)
index = torch.tensor([True, True, False, False])
t[index]= torch.tensor([1, 1])
t

tensor([ 1,  1, 12, 13])

## PPR HOMEWORK

In [3]:
indptr = torch.tensor([0, 1, 2, 3])
indices = torch.tensor([1, 2, 0])
degree = torch.tensor([1, 1, 1])
g = (indptr, indices, degree)


def approx_ppr(g_, a_, t_, e_):
    indptr_, indices_, degree_ = g_
    num_nodes = degree_.size(-1)

    # initialize
    p = torch.zeros(num_nodes)
    r = torch.zeros(num_nodes)
    r[t_] = a_

    threshold = a_ * e_ * degree_
    while True:
        print('p: ', p, ' r: ', r)
        mask = r > threshold
        if mask.sum() == 0:
            break

        # update
        p[mask] += r[mask]
        m = (1 - a_) * r[mask] / degree_[mask]
        r[mask] = 0

        # can be optimized by using scatter()
        v_idx = mask.nonzero(as_tuple=False).view(-1)
        for i, v in enumerate(v_idx):
            u_idx = indices_[indptr_[v]: indptr_[v+1]]
            r[u_idx] += m[i]

    return p


ppr_score = approx_ppr(g, 0.5, 0, 1e-5)
ppr_score

p:  tensor([0., 0., 0.])  r:  tensor([0.5000, 0.0000, 0.0000])
p:  tensor([0.5000, 0.0000, 0.0000])  r:  tensor([0.0000, 0.2500, 0.0000])
p:  tensor([0.5000, 0.2500, 0.0000])  r:  tensor([0.0000, 0.0000, 0.1250])
p:  tensor([0.5000, 0.2500, 0.1250])  r:  tensor([0.0625, 0.0000, 0.0000])
p:  tensor([0.5625, 0.2500, 0.1250])  r:  tensor([0.0000, 0.0312, 0.0000])
p:  tensor([0.5625, 0.2812, 0.1250])  r:  tensor([0.0000, 0.0000, 0.0156])
p:  tensor([0.5625, 0.2812, 0.1406])  r:  tensor([0.0078, 0.0000, 0.0000])
p:  tensor([0.5703, 0.2812, 0.1406])  r:  tensor([0.0000, 0.0039, 0.0000])
p:  tensor([0.5703, 0.2852, 0.1406])  r:  tensor([0.0000, 0.0000, 0.0020])
p:  tensor([0.5703, 0.2852, 0.1426])  r:  tensor([0.0010, 0.0000, 0.0000])
p:  tensor([0.5713, 0.2852, 0.1426])  r:  tensor([0.0000, 0.0005, 0.0000])
p:  tensor([0.5713, 0.2856, 0.1426])  r:  tensor([0.0000, 0.0000, 0.0002])
p:  tensor([0.5713, 0.2856, 0.1428])  r:  tensor([0.0001, 0.0000, 0.0000])
p:  tensor([0.5714, 0.2856, 0.1428]) 

tensor([0.5714, 0.2857, 0.1429])

## PPR Experiments (power-iter vs local-push)

In [12]:
import torch
from collections import defaultdict
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
import numpy as np


def power_iter_ppr(P_w, target_id_, alpha_, epsilon_, max_iter, unweighted_degree_):
    num_nodes = P_w.size(0)
    s = torch.zeros(num_nodes)
    s[target_id_] = 1
    s = s.view(-1, 1)

    x = s.clone()
    num_push = 0
    for i in range(max_iter):
        x_last = x
        x = alpha_ * s + (1 - alpha_) * (P_w @ x)
        # total num of operations
        x_last_nnz_index = x_last.view(-1).nonzero(as_tuple=False).view(-1)
        num_push += unweighted_degree_[x_last_nnz_index].sum()
        # check convergence, l1 norm
        if (abs(x - x_last)).sum() < num_nodes * epsilon_:
            print(f'power-iter      Iterations: {i}, Total Push Operations: {num_push.item()},'
                  f' NNZ: {(x.view(-1) > 0).sum()}')
            return x.view(-1), num_push.item()

    print(f'Failed to converge with tolerance({epsilon_}) and iter({max_iter})')
    return x.view(-1), num_push.item()


def standard_local_push_ppr(g_, target_id_, alpha_, epsilon_, beta_=0., gamma_=1., max_degree_=-1, drop_coe_=0):
    num_nodes = g_['weighted_degree'].size(-1)
    p = torch.zeros(num_nodes)
    r = torch.zeros(num_nodes)
    r[target_id_] = 1

    iterations = num_push = 0
    num_affected_nodes = num_affected_edges = 0
    threshold = epsilon_ * g_['weighted_degree']
    drop_threshold = drop_coe_ * threshold
    bias = beta_ * threshold

    while True:
        v_mask = r > threshold
        # if iterations == 15:
        #     break
        if v_mask.sum() == 0:
            break

        # drop nodes below specific threshold
        # print(((r < drop_threshold) & (r > 0)).sum().item())
        r[(r < drop_threshold) & (r > 0)] = 0

        v_idx = v_mask.nonzero(as_tuple=False).view(-1)
        for i, v in enumerate(v_idx):
            start, end =  g_['indptr'][v], g_['indptr'][v+1]
            num_degree = end - start
            if max_degree_ == -1 or num_degree <= max_degree_:
                ptr = torch.arange(start, end)
            else:
                ptr = torch.randperm(num_degree)[:max_degree_] + start
                num_affected_nodes += 1
                num_affected_edges += num_degree - max_degree_

            u_idx = g_['indices'][ptr]
            u_weights = g_['edge_weights'][ptr]

            while r[v] > threshold[v]:
                # update source node
                if beta_ == 0:
                    p[v] += alpha_ * r[v]
                    m_v = (1 - alpha_) * gamma_ * r[v]
                    r[v] = (1 - alpha_) * (1 - gamma_) * r[v]
                else:
                    p[v] += alpha_ * (r[v] - bias[v])
                    m_v = (1 - alpha_) * (r[v] - bias[v])
                    r[v] = bias[v]

                # batch update neighbors
                r[u_idx] += m_v * (u_weights / u_weights.sum())

                num_push += end - start

        iterations += 1

    return p, iterations, num_push.item(), num_affected_nodes, num_affected_edges


def uniform_local_push_ppr(g_, target_id_, alpha_, epsilon_, gamma_=1.):
    num_nodes = g_['weighted_degree'].size(-1)
    p = torch.zeros(num_nodes)
    r = torch.zeros(num_nodes)
    r[target_id_] = 1

    iterations, num_push = 0, 0
    threshold = epsilon_ * g_['weighted_degree']
    while True:
        v_mask = r > threshold
        if v_mask.sum() == 0:
            break

        v_idx = v_mask.nonzero(as_tuple=False).view(-1)
        for i, v in enumerate(v_idx):
            # update source node
            p[v] += alpha_ * r[v]
            temp = (1 - alpha_) * r[v]
            r[v] = (1 - gamma_) * temp
            m_v = gamma_ * temp / g_['weighted_degree'][v]

            # batch update neighbors
            start, end =  g_['indptr'][v], g_['indptr'][v+1]
            u_idx = g_['indices'][start: end]
            u_weights = g_['edge_weights'][start: end]
            r[u_idx] += m_v * u_weights

            num_push += end - start

        iterations += 1

    return p, iterations, num_push.item(), 0, 0


def batch_local_push_ppr(g_, target_id_, alpha_, epsilon_, gamma_=1., max_degree_=-1, drop_coe_=0):
    num_nodes = g_['weighted_degree'].size(-1)
    p = torch.zeros(num_nodes)
    r = torch.zeros(num_nodes)
    r[target_id_] = 1

    iterations = num_push = 0
    num_affected_nodes = num_affected_edges = 0
    threshold = epsilon_ * g_['weighted_degree']
    drop_threshold = drop_coe_ * threshold

    num_p = 0

    while True:
        v_mask = r > threshold
        if v_mask.sum() == 0:
            break

        # drop nodes below specific threshold
        r[(r < drop_threshold) & (r > 0)] = 0

        v_idx_ = v_mask.nonzero(as_tuple=False).view(-1)
        # while True:
        #     v_mask_ = r[v_idx] > threshold[v_idx]
        #     if v_mask_.sum() == 0:
        #         break
        #     v_idx_ = v_idx[v_mask_]

        # print(v_idx_.numel())
        num_p += v_idx_.numel()

        # batch update source nodes
        p[v_idx_] += alpha_ * r[v_idx_]
        m = (1 - alpha_) * r[v_idx_]
        r[v_idx_] = (1 - gamma_) * m
        m = gamma_ * m

        for i, v in enumerate(v_idx_):
            start, end =  g_['indptr'][v], g_['indptr'][v+1]
            num_degree = end - start
            if max_degree_ == -1 or num_degree <= max_degree_:
                ptr = torch.arange(start, end)
            else:
                ptr = torch.randperm(num_degree)[:max_degree_] + start
                num_affected_nodes += 1
                num_affected_edges += num_degree - max_degree_
            # batch update neighbors
            u_idx = g_['indices'][ptr]
            u_weights = g_['edge_weights'][ptr]
            r[u_idx] += m[i] * u_weights / u_weights.sum()

            num_push += end - start

        iterations += 1

    print('Total Num P', num_p)
    return p, iterations, num_push.item(), num_affected_nodes, num_affected_edges

In [7]:
from ogb.nodeproppred import PygNodePropPredDataset

"""Graph Loading"""
dataset = PygNodePropPredDataset(name='ogbn-products', root='/data/gangda/ogb', transform=T.ToSparseTensor())
# dataset = Planetoid(root='/data/gangda/pyg', name='Cora', pre_transform=T.ToSparseTensor())
data = dataset[0]
unweighted_degree = data.adj_t.sum(dim=0).to(torch.int)
coo_adj_edge_weight = torch.rand(data.num_edges)
# coo_adj_edge_weight = torch.ones(data.num_edges)
data.adj_t.set_value_(coo_adj_edge_weight, layout='csc')

# local-push
indptr, indices, value = data.adj_t.csc()
degree = data.adj_t.sum(dim=0).to(torch.float)
g = {
    'indptr': indptr,
    'indices': indices,
    'edge_weights': value,
    'weighted_degree': degree,
}

# power-iteration
norm_adj_t = data.adj_t * degree.pow(-1).view(1, -1)

g, norm_adj_t

({'indptr': tensor([        0,       174,       263,  ..., 123718233, 123718242,
          123718280]),
  'indices': tensor([    384,    2412,    7554,  ..., 1787657, 1864057, 2430488]),
  'edge_weights': tensor([0.8107, 0.1535, 0.7825,  ..., 0.8139, 0.9815, 0.1251]),
  'weighted_degree': tensor([84.9130, 48.3407, 16.8389,  ..., 27.0977,  4.8304, 20.9754])},
 SparseTensor(row=tensor([      0,       0,       0,  ..., 2449028, 2449028, 2449028]),
              col=tensor([    384,    2412,    7554,  ..., 1787657, 1864057, 2430488]),
              val=tensor([0.0343, 0.0047, 0.0039,  ..., 0.0106, 0.0138, 0.0709]),
              size=(2449029, 2449029), nnz=123718280, density=0.00%))

Nodes affected ratio of different max degree
- 1000 0.1%
- 350 1%
- 200 3%
- 150 6%
- 100 12%
- 80 18%
- 60 24%
- 50 30%

In [1142]:
(unweighted_degree > 60).sum() / data.num_nodes

tensor(0.2585)

In [910]:
"""Deprecated Sampling"""
# sampled local-push
adj = data.adj_t.t()
adj = adj.sample_adj(torch.arange(data.num_nodes), 150, replace=False)[0]
_indptr, _indices, _value = adj.csr()
_degree = adj.sum(dim=1).to(torch.float)
g_sampled = {
    'indptr': _indptr,
    'indices': _indices,
    'edge_weights': _value,
    'weighted_degree': _degree,
}
g_sampled

{'indptr': tensor([    0,     3,     6,  ..., 10530, 10534, 10538]),
 'indices': tensor([ 633, 1862, 2582,  ...,  598, 1473, 2706]),
 'edge_weights': tensor([0.2110, 0.0817, 0.7195,  ..., 0.5115, 0.7540, 0.7701]),
 'weighted_degree': tensor([1.0122, 0.9240, 2.2092,  ..., 0.2975, 2.3681, 2.1038])}

In [13]:
import time

"""Testing"""
lazy_alpha = 0.3
alpha = (2 * lazy_alpha) / (1 + lazy_alpha)
epsilon = 1e-6

max_degree = 1000
drop_coe = 0.8

top_k = 100
num_source = 10

# Modify your test:
test_list = ['standard', 'batch']

def get_approx_ppr(key_, target_id_):
    ppr_func_dict = {
        'standard': standard_local_push_ppr,  # PPRGo, r_v = 0
        'uniform': uniform_local_push_ppr,  # same as 'standard'
        'batch': batch_local_push_ppr,  # update all source nodes simultaneously
        'bias': standard_local_push_ppr,  # MAPPR, 0 < r_v < threshold
        'lazy': standard_local_push_ppr,  # original local-push
        'lazy-uniform': uniform_local_push_ppr,
        'lazy-batch': batch_local_push_ppr,
        'clipped': standard_local_push_ppr,  # 'standard' ppr on clipped graph
        'clipped-lazy': standard_local_push_ppr,
        'clipped-batch': batch_local_push_ppr,
        'drop': standard_local_push_ppr,
        'drop-batch': batch_local_push_ppr,
    }
    kwargs = {
        'alpha_': lazy_alpha if 'lazy' in key_ else alpha,
        'gamma_': 0.5 if 'lazy' in key_ else 1.,
        'epsilon_': epsilon,
    }
    if 'bias' in key_:
        kwargs['beta_'] = 0.5
    if 'clipped' in key_:
        kwargs['max_degree_'] = max_degree
    if 'drop' in key_:
        kwargs['drop_coe_'] = drop_coe
    return ppr_func_dict[key_](g, target_id_, **kwargs)

total_base_p = 0.
total_concur, total_err, total_push = defaultdict(int), defaultdict(float), defaultdict(int)
total_clipped_nodes, total_clipped_edges = defaultdict(int), defaultdict(int)
total_time = defaultdict(float)

source_nodes = torch.randperm(data.num_nodes)[:num_source]
for epoch, target_id in enumerate(source_nodes):
    print(f'\n########## Iter {epoch+1} ##########')

    tik = time.time()
    base_p, base_num_push = power_iter_ppr(norm_adj_t, target_id, alpha, 1e-10, 100, unweighted_degree)
    total_time['power-iter'] += time.time() - tik

    total_base_p += base_p
    total_push['power-iter'] += base_num_push
    _, base_top_k = torch.sort(base_p, descending=True)

    for key in test_list:
        tik = time.time()
        approx_p, approx_num_iter, approx_num_push, num_clipped_nodes, num_clipped_edges = get_approx_ppr(key, target_id)
        total_time[key] += time.time() - tik

        print(f'{key:15s} Iterations: {approx_num_iter}, Total Push Operations: {approx_num_push}, NNZ: {(approx_p > 0).sum()}, Clipped Nodes(Edges): {num_clipped_nodes}({num_clipped_edges})')

        total_push[key] += approx_num_push
        total_err[key] += (abs(approx_p - base_p)).sum().item()
        _, approx_top_k = torch.sort(approx_p, descending=True)
        total_concur[key] += np.intersect1d(base_top_k[:top_k], approx_top_k[:top_k]).shape[0]

        total_clipped_nodes[key] += num_clipped_nodes
        total_clipped_edges[key] += num_clipped_edges

# print overall results
print(f'\n\n########## Results ##########')
print(f'Parameters: lazy_alpha={lazy_alpha}, alpha={alpha:.3f}, epsilon={epsilon:.1e}, max_degree={max_degree}, drop_coe={drop_coe}')

print(f'\nAvg Push Operations:')
for k, val in total_push.items():
    print(f'\t{k}: {val/num_source:.0f}')

print(f'\nAvg Clipped Nodes(Edges):')
for k, val in total_clipped_nodes.items():
    print(f'\t{k}: {val/num_source:.0f}({total_clipped_edges[k]/num_source:.0f})')

print(f'\nPrecision Top-{top_k}:')
for k, val in total_concur.items():
    print(f'\t{k}: {val/(top_k * num_source):.3f}')

print(f'\nmean power-iter ppr: {total_base_p.mean().item()/num_source: .3e}'
      f'\nMean Absolute Error:')
for k, val in total_err.items():
    print(f'\t{k}: {val/(data.num_nodes*num_source):.3e}')

print(f'\nAvg Run Time:')
for k, val in total_time.items():
    print(f'\t{k}: {val/num_source:.3f}s')


########## Iter 1 ##########
power-iter      Iterations: 9, Total Push Operations: 699958434, NNZ: 2338909
standard        Iterations: 9, Total Push Operations: 191441, NNZ: 2690, Clipped Nodes(Edges): 0(0)
Total Num P 5157
batch           Iterations: 20, Total Push Operations: 245324, NNZ: 2663, Clipped Nodes(Edges): 0(0)

########## Iter 2 ##########
power-iter      Iterations: 10, Total Push Operations: 779120479, NNZ: 2338909
standard        Iterations: 11, Total Push Operations: 116896, NNZ: 1738, Clipped Nodes(Edges): 0(0)
Total Num P 4539
batch           Iterations: 12, Total Push Operations: 154494, NNZ: 1735, Clipped Nodes(Edges): 0(0)

########## Iter 3 ##########
power-iter      Iterations: 9, Total Push Operations: 718832620, NNZ: 2338909
standard        Iterations: 8, Total Push Operations: 160632, NNZ: 2042, Clipped Nodes(Edges): 0(0)
Total Num P 3188
batch           Iterations: 10, Total Push Operations: 175921, NNZ: 2015, Clipped Nodes(Edges): 0(0)

########## Iter 4 #

## PPRGO (close to C implementation)

In [ ]:
def _calc_ppr_node(target_id_, indptr_, indices_, degree_, weights_, alpha_, epsilon_):
    alpha_eps = alpha_ * epsilon_
    f32_0 = 0.
    p = {target_id_: f32_0}
    r = {target_id_: alpha_}
    q = [target_id_]
    while len(q) > 0:
        unode = q.pop()

        res = r[unode] if unode in r else f32_0
        if unode in p:
            p[unode] += res
        else:
            p[unode] = res
        r[unode] = f32_0
        for vnode in indices_[indptr_[unode]:indptr_[unode + 1]]:
            _val = (1 - alpha_) * res * weights_[unode] / degree_[unode]
            if vnode in r:
                r[vnode] += _val
            else:
                r[vnode] = _val

            res_vnode = r[vnode] if vnode in r else f32_0
            if res_vnode >= alpha_eps * degree_[vnode]:
                if vnode not in q:
                    q.append(vnode)

    return list(p.keys()), list(p.values())

# PPRGO Implementation, needs numba compiler acceleration
# Root nodes level parallelization
_calc_ppr_node(0, indptr.tolist(), indices.tolist(), degree.tolist(), value.tolist(), 0.15, 1e-3)

In [9]:
def local_push_ppr(g_, target_id_, alpha_, epsilon_, max_degree_=-1, drop_coe_=0.5):
    num_nodes = g_['weighted_degree'].size(-1)
    p = torch.zeros(num_nodes)
    r = torch.zeros(num_nodes)
    r[target_id_] = 1

    threshold = epsilon_ * g_['weighted_degree']
    drop_threshold = drop_coe_ * threshold

    while True:
        v_mask = r > threshold
        if v_mask.sum() == 0:
            break

        # drop nodes below specific threshold
        r[(r < drop_threshold) & (r > 0)] = 0

        v_idx = v_mask.nonzero(as_tuple=False).view(-1)
        for i, v in enumerate(v_idx):
            start, end = g_['indptr'][v], g_['indptr'][v + 1]
            num_degree = end - start
            if max_degree_ == -1 or num_degree <= max_degree_:
                ptr = torch.arange(start, end)
            else:
                ptr = torch.randperm(num_degree)[:max_degree_] + start

            u_idx = g_['indices'][ptr]
            u_weights = g_['edge_wewights'][ptr]

            # update source node
            p[v] += alpha_ * r[v]
            m_v = (1 - alpha_) * r[v]
            r[v] = 0
            # batch update neighbors
            r[u_idx] += m_v * (u_weights / u_weights.sum())

    return p